# Markov prediction
In this notebook we will implement (arguably) the most simple form of sequence generation: a [bigram](https://en.wikipedia.org/wiki/Bigram) model where each subsequent token (character) is generated by sampling from the conditional probability distribution $P(X_t \mid X_{t-1}) \sim Multinomial_{X_{t-1}}(N_{tokens})$, i.e. every character (token) has an associated probility table for what the next character (token) will be. 'Training' proceeds through simple counting of observed character pairs.

We start with a very minimal implementation and then proceed to introduce some boilerplate for data loading, model forwarding, and training. Whereas this is not strictly necesarry for this model, it makes comparisons with later more complex models more straightforward. More importantly, our advanced implementation will be generalized to [n-grams](https://en.wikipedia.org/wiki/N-gram), i.e. higher order Markov chains. For example, an n-gram model of order two is $P(X_t \mid X_{t-1}, X_{t-2}) \sim Multinomial_{X_{t-1}, X_{t-2}}(N_{tokens})$. This will allow us to explore the limits of conditional probability lookup tables (e.g. in terms of performance and number of parameters).

In [ ]:
# all dependencies for the entire notebook
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import RandomSampler, random_split

# Data

We use the tiny Shakespeare dataset to train a character level Markov model to predict text that looks very little like Shakespeare. All data is in one text file, which we download below.

In [ ]:
# Download the tiny shakespeare dataset
!wget -nc https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# Minimal bigram implementation
Below is a very minimal implementation of generating a text sample using Markov prediction: every next token is sampled according to a probability based only on the previous token. 'Training' consists of counting observed character pairs. We don't calculate a loss and model evaluation is based on intuition: whether or not the generated text feels like 'real' text.

In [ ]:
with open('input.txt') as fh:
    data = fh.read()

# create a sorted list of unique characters
chars = sorted(list(set(data)))
# count how many 'tokens'
vocab_size = len(chars)
# map characters to integers
encode = { ch:i for i,ch in enumerate(chars) }
# map integers to characters
decode = { i:ch for i,ch in enumerate(chars) }

# initialize counts with ones for automatic laplace/additive smoothing: https://en.wikipedia.org/wiki/Additive_smoothing
counts = torch.ones(vocab_size, vocab_size)

# 'train' (i.e. count) for n steps (i.e. character pairs)
n_train_chars = len(data) - 1
for i in trange(n_train_chars):
    token_i = encode[data[i]]
    token_j = encode[data[i+1]]
    counts[token_i][token_j] += 1

# divide by row sums to get transition probabilities
probs = counts / counts.sum(dim=1, keepdim=True)

# start our sample with token 0 (i.e. the newline character \n)
sample = [0]
sample_n_tokens = 256

# for n steps, sample a token from the distribution belonging the last token in the current sample and add it to the sample
for _ in range(sample_n_tokens):
    last_token = sample[-1]
    next_token_probs = probs[last_token]
    next_token = torch.multinomial(next_token_probs, num_samples=1)
    sample.append(next_token.item())

# decode sample tokens into characters
decoded_sample = ''.join(decode[token] for token in sample)
print(f'Sample:\n{decoded_sample}\n')

# plot observed transition probabilities
plt.imshow(probs)

### Exercise 1

Experiment with different numbers of training steps (by setting `n_train_chars` in the code above), e.g. 100, 1000, or 10,000. When do you start seeing some structure appearing in the generated samples? What about if you 'train' on the entire dataset? (i.e. `n_train_chars = len(data) - 1`)?

# Advanced n-gram implementation

Clearly bigrams pick up some structure from a large enough text, but are not powerful enough to generate realistic sentences. Below we implement the same counting principle for n-gram models. To do so, we create a character level tokenizer, a dataset class that returns tensors of a certain order, and a model class that takes care of our most commonly used procedures (e.g. parameter initialization, prediction, calculating a loss score, and generating novel text). These main building blocks (tokenizer, dataset, model) look very similar to what we will use in the rest of the week when we build more advanced models based on neural networks.

## Data

In [ ]:
class CharacterTokenizer:
    """Character level tokenizer that enumerates unique characters in a training text"""
    def __init__(self, encoding_dict: dict[str, int]=None):
        if encoding_dict is None:
            self.encoding_dict = dict()
        else:
            self.encoding_dict = encoding_dict

    def __repr__(self):
        return f'CharacterTokenizer(vocab_size={self.vocab_size})'

    @property
    def decoding_dict(self) -> dict[int, str]:
        """Decoding dict is implemented as property to automatically sync with changed encoding dict"""
        return {token:char for char,token in self.encoding_dict.items()}

    @property
    def vocab_size(self) -> int:
        return len(self.encoding_dict)

    def get_vocab(self) -> dict[str, int]:
        return self.encoding_dict

    def train(self, data: str) -> None:
        """Train on a piece of text by enumerating unique characters"""
        chars = sorted(set(data))
        self.encoding_dict = {char:token for token,char in enumerate(chars)}

    def encode(self, data: str) -> list[int]:
        """Convert text to tokens"""
        return [self.encoding_dict.get(char, -1) for char in data]

    def decode(self, tokens: list[int]) -> str:
        """Convert tokens to text"""
        return ''.join(self.decoding_dict.get(token, '<unk>') for token in tokens)

class CharacterDataset:
    def __init__(self, data: str, tokenizer: CharacterTokenizer, order: int=1):
        assert order > 0, 'Order must be > 1'
        self.data = data
        self.tokenizer = tokenizer
        self.vocab_size = len(tokenizer.get_vocab())
        self.order = order

    def __repr__(self):
        n_chars = len(self.data)
        vocab_size = self.vocab_size
        order = self.order
        return f'CharacterDataset({n_chars=}, {vocab_size=}, {order=})'

    @classmethod
    def from_textfile(cls, filename: str, order: int=1) -> 'CharacterDataset':
        """
        Constructor method that builds a character dataset from a text file.
        Text is 'tokenized' by splitting into characters.
        """
        tokenizer = CharacterTokenizer()
        with open(filename, 'r') as fh:
            data = fh.read()
            tokenizer.train(data)
            return cls(data, tokenizer, order=order)

    def __len__(self) -> int:
        return len(self.data) - self.order

    def __getitem__(self, idx: int) -> torch.tensor:
        # grab a chunk of n=order + 1 characters from the data
        chunk = self.data[idx:idx + self.order + 1]
        # encode every character to an integer
        tokens = self.tokenizer.encode(chunk)
        # convert to tensor
        tokens = torch.tensor(tokens, dtype=torch.long)
        # return relevant tokens based on indices
        x = tokens[:-1]
        y = tokens[-1:]
        return x,y

dataset = CharacterDataset.from_textfile('./input.txt')
len(dataset)

## Model
Below we generalize the bigram counting model to work on n-grams of arbitrary order n. We create a class to wrap parameters and add functionality for 'forwarding' the model (i.e. calculating next token probabilities), calculating a loss score, and generating token samples.

In [ ]:
class NGramModel(nn.Module):
    """Very simple model for next character prediction by counting observed n-grams"""
    def __init__(self, vocab_size, order=1):
        assert order > 0, 'Order must be > 1'
        super().__init__()
        self.order = order
        # counting character pairs in a vocab_size ** (order + 1) table, initializing to 1 implements laplace smoothing
        self.counts = torch.ones([vocab_size] * (order + 1))

    def __repr__(self) -> str:
        order = self.order
        return f'NGramModel({order=})'

    @property
    def probs(self) -> torch.tensor:
        """Normalize counts to probabilities by dividing over row sums"""
        return self.counts / self.counts.sum(dim=-1, keepdim=True)

    def update_counts(self, data_batch: torch.Tensor) -> None:
        """Training the model is equal to counting observed occurences"""
        for input,target in zip(*data_batch):
          self.counts[[*input, target]] += 1

    def forward(self, idx: torch.tensor, targets: torch.tensor=None) -> torch.tensor:
        probs = self.probs[[*idx.T]]
        loss = None if targets is None else F.cross_entropy(probs, targets.view(-1))
        return probs,loss

    def generate(self, sample_length: int=256) -> list[int]:
        """Generate text by sampling from P(next_token | previous_n_tokens) where n_tokens is determined by the 'order' of the markov chain"""
        idx = torch.zeros(self.order, dtype=torch.long)
        for _ in trange(sample_length, desc='Sampling'):
            probs,_ = model(idx[-self.order:][None])
            new_idx = torch.multinomial(probs[0], num_samples=1)
            idx = torch.hstack([idx, new_idx])

        return idx.tolist()

dataset = CharacterDataset.from_textfile('./input.txt', order=3)
model = NGramModel(vocab_size = dataset.vocab_size, order = dataset.order)

print(dataset)
print(model)

# Show a sample from an untrained model
sample = model.generate(sample_length=64)
print(dataset.tokenizer.decode(sample))

## Training
Note that whereas we can call this 'training', all the code below does is count character pairs (triples, etc...) and update the pair count table of the model.

In [ ]:
n_train_steps = 10_000
batch_size = 100
order = 2
train_frac = 0.8

dataset = CharacterDataset.from_textfile('./input.txt', order=order)
train_dataset, validation_dataset = random_split(dataset, (train_frac, 1 - train_frac)) 

train_dataloader = DataLoader(
    dataset=train_dataset,
    sampler=RandomSampler(train_dataset, num_samples=n_train_steps * batch_size),
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    dataset=validation_dataset,
    sampler=RandomSampler(validation_dataset, replacement=True), # test dataset is relatively small so we might need to sample more than once
    batch_size=batch_size
)

validation_dataloader = iter(validation_dataloader)

train_loss = []
test_loss = []

model = NGramModel(dataset.vocab_size, dataset.order)

for i, train_batch in enumerate(tqdm(train_dataloader, desc='Training')):
    model.update_counts(train_batch)
    if i % 10 == 0:
        test_input, test_target = next(validation_dataloader)
        test_loss.append(model(test_input, test_target)[1])
        train_input, train_target = train_batch
        train_loss.append(model(train_input, train_target)[1])

plt.plot(train_loss, label='train loss')
plt.plot(test_loss, label='test loss')
plt.legend()

## Evaluation
The model has learned some structure, e.g. there are probably a bit more newlines and a colon is always followed by a newline.

In [ ]:
sample = model.generate()
print(dataset.tokenizer.decode(sample))

### Exercise 2
Fit n-gram models of order 1, 2, and 3, note the training and test loss, and evaluate a few generated samples. What do you observe? Does an n-gram model of order 3 result in reasonable text?

### Exercise 3
How many parameters do you need for an n-gram model of order 4? What do you expect the training loss will be (extrapolate from the results you got in excercise 2)?